In [1]:
import pandas as pd
import numpy as np
import csv
from vzorci_in_pomozne_definicije import *

# Zdaj hočemo narediti podatkovno bazo z merganjem dveh dataframov

In [2]:
slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani = {'set': {"VAN" : "PVAN", "SLDC": "PHED", "XCLE": "CED", "XICE": "CEI", "RMB1": "FMB1", "XDND" : "AFR", "2E": "LEB", "2U": "2ED", "1E": "LEA", "PO2": "P02", "3ED":"FBB", "3E":"3ED"}}

osnovni_podatki_o_kartah = pd.read_csv("karte.csv").replace(slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani)
dodatni_podatki_o_kartah = pd.read_csv("specificne_lastnosti_kart.csv")
dodatni_podatki_o_setih = pd.read_csv("seti.csv").replace(slovar_za_popravljanje_napacnih_podatkov_s_spletne_strani)

print(dodatni_podatki_o_setih)

podatkovna_baza = pd.merge(osnovni_podatki_o_kartah, dodatni_podatki_o_kartah)
podatkovna_baza = pd.merge(podatkovna_baza, dodatni_podatki_o_setih)

# koliko je sploh smiselno družiti take podatke med sabo?
# V resnici ni čisto jasno

%reset_selective -f osnovni_podatki_o_kartah
%reset_selective -f dodatni_podatki_o_kartah
%reset_selective -f dodatni_podatki_o_setih

     set               polno_ime  st_kart datum_izida
0    RTR       Return to Ravnica      274  2012-10-05
1    ZEN                Zendikar      269  2009-10-02
2    BRB   Battle Royale Box Set      136  1999-11-12
3    SNC  Streets of New Capenna      510  2022-04-29
4    NCC   New Capenna Commander      447  2022-04-29
..   ...                     ...      ...         ...
221  S00            Starter 2000       20  2000-04-01
222  POR                  Portal      256  1997-05-01
223  P02       Portal Second Age      165  1998-06-24
224  PTK   Portal Three Kingdoms      180  1999-07-06
225  ARC               Archenemy      150  2010-06-18

[226 rows x 4 columns]


Zdaj malo popravimo tabelo s funckijami, ki smo jih napisali.

In [3]:
podatkovna_baza["cardtype"] = podatkovna_baza["cardtype"].apply(lambda x: popravi_cardtype_aftermath(x[1:-1].replace("'", "").split(', ')) if type(x) == str else x)
podatkovna_baza["subtype"] = podatkovna_baza["subtype"].apply(lambda x: popravi_subtype_adventure(x[1:-1].replace("'", "").split(', ')) if type(x) == str else x)
podatkovna_baza["supertype"] = podatkovna_baza["supertype"].apply(lambda x: x[1:-1].replace("'", "").split(', ') if type(x) == str else x)
podatkovna_baza["reserved_list"] = podatkovna_baza["reserved_list"].apply(lambda x: x == "Yes")
# Moram ugotoviti, kako se spodnje 3 vrstice zapiše v enem koraku
podatkovna_baza["all_time_low_datum"] = podatkovna_baza["all_time_low_datum"].apply(lambda x: pd.to_datetime(pretvori_datum_v_datetime(x)) if type(x) == str else x)
podatkovna_baza["all_time_high_datum"] = podatkovna_baza["all_time_high_datum"].apply(lambda x: pd.to_datetime(pretvori_datum_v_datetime(x)) if type(x) == str else x)
podatkovna_baza["datum_izida"] = podatkovna_baza["datum_izida"].apply(lambda x: pd.to_datetime(pretvori_datum_v_datetime(x)) if type(x) == str else x)

podatkovna_baza["barva"] = podatkovna_baza.apply(lambda x: popravi_barvo_karte(str(x.barva), str(x.oracle_text), str(x.cardtype)), axis = 1)

seznam_stolpcev_cen = ["povprecna_cena", "povprecna_cena_foil", "povprecje_eu", "all_time_low", "all_time_high"]
for cena in seznam_stolpcev_cen:
    podatkovna_baza[cena] = podatkovna_baza[cena].apply(lambda x: (float(x[1:])) if (type(x) != float) else x).replace("nan","NaN") # Tole je seveda treba popraviti, da ne bo grdo

#podatkovna_baza["cardtype"] = podatkovna_baza["cardtype"].apply(lambda x: vsebovanost_problematicnih_karte_na_podlagi_seznama(x[1:-1].split(',')) if type(x) == str else x)

podatkovna_baza.to_json("popolni_podatki_o_kartah.json", orient='records', indent=4)

# Zdaj je treba narediti povezovalne tabele

In [4]:
def ustvari_povezave(indeks, seznam, seznam_povezav):
    if type(seznam) != list:
        return (indeks, None)
    for element in seznam:
        #print((indeks, element))
        seznam_povezav.append((indeks, element))

def ustvari_povezovalno_tabelo_glede_na_seznam(ime_id_stolpca, ime_seznam_stolpca):
    seznam_povezav = []
    """
    To ustvari povezovalno tabelo glede na podana seznam in indeks
    """
    
    #podatkovna_baza.apply(lambda x: print(x.id_karte, x.cardtype), axis = 1)
    podatkovna_baza.apply(lambda x: ustvari_povezave(x[ime_id_stolpca], x[ime_seznam_stolpca], seznam_povezav), axis = 1)

    return(seznam_povezav)
    
#ustvari_povezovalno_tabelo_glede_na_seznam("id_karte","cardtype")
    

In [5]:
for tip in ["cardtype","subtype","supertype"]:
    with open(f"povezovalna_{tip}.csv", "w", encoding="utf-8") as csvfile:
        seznam_tuplov = ustvari_povezovalno_tabelo_glede_na_seznam("id_karte",f"{tip}")
        writer = csv.writer(csvfile)
        writer.writerow(["id_karte", f"{tip}"])
        for vrstica in seznam_tuplov:
            writer.writerow(vrstica)